In [2]:
// :dep tch = { version = "0.1.7", path = "../../LaurentMazare/tch-rs" }
// use tch::{Tensor, IndexOp, NewAxis};
// let x: Tensor = Tensor::of_slice(&[1, 2, 3, 4, 5, 6]).reshape(&[2, 3]);
// x.print();
// let y: Tensor = x.i((.., 0, NewAxis));
// y.print();
// x.print();
// println!("{:?}", y.size());

# Quantile regression

Python code is [here](https://gist.github.com/taku-y/471b7eae5ef85badb1ddcfb389982bd6).

In [3]:
:dep tch = { version = "0.1.7", path = "../../LaurentMazare/tch-rs" }
// :dep plotters = { git = "https://github.com/38/plotters", features = ["evcxr"] }
:dep plotters = { path = "../../38/plotters", features = ["evcxr"] }
:dep qr = { path = "./" }

In [4]:
use tch::{nn, nn::ModuleT, Tensor, Kind};
use qr::{create_model, train_qr_model, load_data};
use plotters::prelude::*;

In [5]:
fn plot_result(xs: &Tensor, ys: &Tensor) -> plotters::evcxr::SVGWrapper {
    evcxr_figure((640, 480), |root| {
    
        // The following code will create a chart context
        let mut chart = ChartBuilder::on(&root)
            .caption("Training data", ("Arial", 20).into_font())
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_ranged(12f64..22f64, -2f64..3f64)?;
    
        chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
            .draw()?;
        
        // Plot prediction
        let n_points = xs.size()[0];
        for j in 0..=6 {
            chart.draw_series(LineSeries::new((0..n_points).map(|i|
                (xs.double_value(&[i, 0]), ys.double_value(&[i, j]))),
                &BLUE
            ));
        }
        
        // Scatter plot
        let (xs, ys) = load_data().unwrap();
        let n_points = xs.size()[0];
        chart.draw_series((0..n_points).map(|i| Circle::new(
            (xs.double_value(&[i]), ys.double_value(&[i])), 3, BLUE.filled()
        )));

        Ok(())
    }).style("width:60%")
}

In [8]:
fn train(cumsum: bool) -> plotters::evcxr::SVGWrapper {
    // Device
    let device = tch::Device::cuda_if_available();
    let vs = nn::VarStore::new(device);
    
    // Initialize model
    let n_inputs = 1i64;
    let n_quants = 7i64;
    let model = create_model(&vs.root(), n_inputs, n_quants, vec![8], cumsum);

    // Training loop
    // the type impl tch::nn::module::ModuleT cannot currently be persisted
    let model = train_qr_model(model, n_quants, device, &vs);
    
    // Prediction
    let xs = Tensor::arange2(10.0, 24.0, 0.2, (Kind::Float, device)).view_(&[-1, 1]);
    let ys = xs.apply_t(&model, false); // train = false in 2nd arg
    println!("{:?}", ys.size());
    
    // Plot prediction and data
    plot_result(&xs, &ys)
}

## Without cumsum

Quantile regression curves are estimated independently, thus the order of regression curves may be inconsistent.

In [12]:
train(false)

Quantiles
 0.1250
 0.2500
 0.3750
 0.5000
 0.6250
 0.7500
 0.8750
[ CPUFloatType{7} ]
iter =     0, loss = 0.36304357647895813
iter =  2000, loss = 0.20935633778572083
iter =  4000, loss = 0.2083345651626587
iter =  6000, loss = 0.2075996696949005
iter =  8000, loss = 0.20779283344745636
iter = 10000, loss = 0.2074453979730606
iter = 12000, loss = 0.2065182477235794
iter = 14000, loss = 0.20544277131557465
iter = 16000, loss = 0.20386028289794922
iter = 18000, loss = 0.20136895775794983
[70, 7]


Training data
 
 
 
-1.5
 
 
 
-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
 
13.0
 
 
 
14.0
 
 
 
15.0
 
 
 
16.0
 
 
 
17.0
 
 
 
18.0
 
 
 
19.0
 
 
 
20.0
 
 
 
21.0
 
 
 
22.0
 
 
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,47 40,60 40,74 40,89 40,104 40,119 40,135 40,152 40,168 40,184 40,200 52,215 64,230 76,243 88,256 100,268 112,278 124,286 136,293 148,299 160,303 172,306 184,307 196,307 208,306 220,304 232,302 244,299 256,295 268,291 280,288 292,285 304,282 316,280 328,279 340,279 352,279 364,280 376,282 388,286 400,289 412,294 424,299 436,305 448,311 460,318 472,325 484,332 496,339 508,346 520,353 532,361 544,368 556,374 568,381 580,387 592,393 604,399 616,405 628,410 640,415 640,420 640,424 640,428 640,432 640,435 640,439 640,440 640,440 640,440 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,43 40,66 40,90 40,113 52,136 64,158 76,178 88,198 100,215 112,231 124,245 136,257 148,268 160,276 172,282 184,286 196,289 208,290 220,289 232,288 244,286 256,283 268,280 280,277 292,274 304,271 316,269 328,267 340,266 352,266 364,267 376,269 388,271 400,274 412,278 424,283 436,288 448,294 460,300 472,306 484,313 496,320 508,326 520,333 532,340 544,346 556,353 568,359 580,365 592,371 604,376 616,381 628,386 640,391 640,395 640,399 640,403 640,407 640,410 640,413 640,416 640,418 640,421 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,44 40,66 40,89 52,111 64,132 76,152 88,171 100,189 112,204 124,219 136,231 148,242 160,250 172,257 184,262 196,266 208,268 220,269 232,269 244,268 256,267 268,265 280,263 292,261 304,260 316,258 328,257 340,257 352,258 364,259 376,261 388,263 400,266 412,270 424,274 436,279 448,284 460,290 472,295 484,301 496,307 508,313 520,319 532,325 544,331 556,337 568,342 580,347 592,352 604,357 616,362 628,366 640,370 640,374 640,378 640,381 640,384 640,387 640,390 640,393 640,395 640,397 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,46 40,65 40,85 52,104 64,122 76,139 88,156 100,171 112,185 124,198 136,209 148,218 160,226 172,233 184,238 196,242 208,244 220,246 232,247 244,247 256,247 268,246 280,245 292,244 304,244 316,244 328,244 340,245 352,246 364,248 376,250 388,253 400,257 412,261 424,265 436,270 448,275 460,281 472,286 484,292 496,298 508,304 520,309 532,315 544,321 556,326 568,331 580,336 592,341 604,346 616,350 628,354 640,358 640,362 640,365 640,368 640,371 640,374 640,377 640,379 640,381 640,384 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,38 40,55 40,73 52,90 64,106 76,122 88,137 100,151 112,164 124,175 136,186 148,195 160,203 172,209 184,214 196,218 208,222 220,224 232,225 244,226 256,227 268,227 280,228 292,228 304,228 316,229 328,230 340,232 352,234 364,236 376,239 388,242 400,246 412,251 424,255 436,260 448,266 460,271 472,277 484,282 496,288 508,294 520,299 532,305 544,311 556,316 568,321 580,326 592,331 604,335 616,340 628,344 640,348 640,351 640,355 640,358 640,361 640,364 640,367 640,369 640,372 640,374 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 52,27 64,27 76,46 88,64 100,81 112,97 124,112 136,125 148,137 160,147 172,157 184,165 196,171 208,177 220,182 232,186 244,189 256,192 268,194 280,197 292,199 304,201 316,203 328,206 340,209 352,212 364,216 376,220 388,225 400,229 412,235 424,240 436,246 448,251 460,257 472,263 484,269 496,275 508,281 520,287 532,293 544,298 556,304 568,309 580,314 592,318 604,323 616,327 628,331 640,335 640,338 640,342 640,345 640,348 640,350 640,353 640,355 640,357 640,360 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,27 40,27 40,2

## With cumsum

Regression curves are cumulative sum of positive values excepting the lowest quantile, so they are always ordered from lower to higer quantiles. See the implementation of the output layer in `create_model()` in `lib.rs`.

In [17]:
train(true)

Quantiles
 0.1250
 0.2500
 0.3750
 0.5000
 0.6250
 0.7500
 0.8750
[ CPUFloatType{7} ]
iter =     0, loss = 0.8314995169639587
iter =  2000, loss = 0.2114652693271637
iter =  4000, loss = 0.20878703892230988
iter =  6000, loss = 0.20742233097553253
iter =  8000, loss = 0.20734447240829468
iter = 10000, loss = 0.2072826772928238
iter = 12000, loss = 0.2068628966808319
iter = 14000, loss = 0.20639674365520477
iter = 16000, loss = 0.20528718829154968
iter = 18000, loss = 0.20336098968982697
[70, 7]


Training data
 
 
 
-1.5
 
 
 
-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
 
13.0
 
 
 
14.0
 
 
 
15.0
 
 
 
16.0
 
 
 
17.0
 
 
 
18.0
 
 
 
19.0
 
 
 
20.0
 
 
 
21.0
 
 
 
22.0
 
 
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,222 40,225 40,227 40,230 40,233 40,237 40,240 40,244 40,248 40,253 40,257 52,262 64,268 76,273 88,277 100,282 112,286 124,290 136,293 148,295 160,297 172,298 184,298 196,299 208,299 220,298 232,298 244,297 256,296 268,296 280,295 292,295 304,295 316,295 328,296 340,297 352,298 364,300 376,303 388,305 400,308 412,312 424,315 436,320 448,324 460,328 472,333 484,338 496,343 508,347 520,352 532,357 544,361 556,366 568,370 580,374 592,378 604,381 616,384 628,387 640,390 640,393 640,395 640,397 640,399 640,400 640,402 640,403 640,404 640,405 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,44 40,72 40,99 40,125 52,149 64,170 76,189 88,206 100,221 112,233 124,243 136,251 148,257 160,262 172,265 184,267 196,269 208,270 220,270 232,270 244,269 256,269 268,268 280,268 292,267 304,267 316,268 328,268 340,269 352,271 364,273 376,275 388,278 400,282 412,286 424,290 436,294 448,299 460,304 472,310 484,315 496,320 508,326 520,331 532,336 544,341 556,346 568,351 580,355 592,359 604,363 616,367 628,370 640,373 640,376 640,378 640,380 640,382 640,384 640,386 640,387 640,388 640,389 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,58 40,88 52,115 64,140 76,163 88,182 100,199 112,213 124,225 136,234 148,242 160,247 172,251 184,254 196,256 208,258 220,258 232,258 244,258 256,257 268,257 280,256 292,256 304,256 316,256 328,257 340,258 352,260 364,261 376,264 388,267 400,270 412,274 424,278 436,283 448,288 460,293 472,298 484,303 496,309 508,314 520,319 532,325 544,330 556,335 568,339 580,344 592,348 604,352 616,355 628,358 640,361 640,364 640,366 640,369 640,371 640,372 640,374 640,375 640,376 640,377 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,55 52,85 64,112 76,136 88,158 100,176 112,192 124,205 136,216 148,224 160,230 172,235 184,239 196,241 208,243 220,244 232,244 244,244 256,244 268,243 280,243 292,243 304,243 316,243 328,244 340,245 352,247 364,249 376,252 388,255 400,258 412,262 424,267 436,271 448,276 460,282 472,287 484,293 496,298 508,304 520,310 532,315 544,320 556,325 568,330 580,334 592,339 604,343 616,346 628,350 640,353 640,355 640,358 640,360 640,362 640,364 640,366 640,367 640,368 640,369 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 52,56 64,85 76,110 88,133 100,153 112,169 124,183 136,194 148,203 160,210 172,216 184,220 196,223 208,225 220,226 232,227 244,227 256,227 268,227 280,227 292,227 304,227 316,228 328,229 340,231 352,233 364,235 376,238 388,241 400,245 412,250 424,254 436,259 448,265 460,270 472,276 484,282 496,288 508,294 520,299 532,305 544,310 556,316 568,320 580,325 592,330 604,334 616,337 628,341 640,344 640,347 640,349 640,352 640,354 640,356 640,357 640,359 640,360 640,361 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 52,27 64,33 76,60 88,84 100,104 112,122 124,137 136,149 148,159 160,168 172,174 184,179 196,184 208,187 220,189 232,191 244,192 256,194 268,195 280,196 292,197 304,199 316,201 328,203 340,206 352,209 364,212 376,216 388,221 400,226 412,231 424,237 436,243 448,249 460,255 472,262 484,268 496,275 508,281 520,288 532,294 544,300 556,305 568,311 580,316 592,320 604,325 616,329 628,333 640,336 640,339 640,342 640,344 640,346 640,348 640,350 640,352 640,353 640,354 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,27 40,27 